In [1]:
import numpy as np 
import pandas as pd
import pickle

In [2]:
# import river modules
from river import datasets
from river import evaluate
from river import metrics
from river import optim
from river import preprocessing
from river import linear_model
from river import drift

#### Drift Detection using HDDM_A

In [3]:
# Dataset
dataset = datasets.synth.Agrawal(
     classification_function=5,
     seed=42
)

# Initializing model
LRmodel = pickle.load(open('../saved_models/classification/model.pkl', 'rb'))

# Initialize HDMM_A drift detector
hddm_a = drift.HDDM_A(drift_confidence=0.05,
                     warning_confidence=0.09,
                     two_sided_test=False)

# Initialise metric
metric = metrics.Accuracy()
print_warning = True
drifts = list()
warnings = list()

for i,data in enumerate(dataset.take(800)):
    # data[0] - features, data[1] - label
    y_pred = LRmodel.predict_one(data[0])
    metric = metric.update(data[1],y_pred)
    # correctly_classified if y_pred == data[1] for checking accuracy
    hddm_a.update(y_pred == data[1])   # Data is processed one sample at a time
    if hddm_a.warning_detected and print_warning:
        # The drift detector indicates after each sample if there is a drift in the data
        print(f'Warning detected at index {i}')
        print_warning = False
        warnings.append(i)
    if hddm_a.drift_detected:
        print(f"Change detected at index {i}")
        print_warning = True
        drifts.append(i)

print(f'{metric}\n')
print(f'Warnings: {warnings}')
print(f'Drifts: {drifts}')

Warning detected at index 189
Change detected at index 637
Warning detected at index 654
Change detected at index 655
Accuracy: 62.75%

Warnings: [189, 654]
Drifts: [637, 655]
